# Atividade 6 -  Seleção de features

Atividade (1.25): realizar a seleção de de características baseada no coeficiente de correlação de Pearson atendendo aos seguintes itens:
<ol>
        <li> Criar uma função chamada feature_selector que irá receber um limiar como parâmetro de entrada e deverá retornar um subconjunto dos dados referente aquelas features cuja correlação está acima do limiar passado; </li> 
        <li> Extrapolar a função get_best_model para abranger os classificadores svm e gxboost;</li> 
        <li> Extrapolar a função get_best_model para abranger a otimização Bayesiana (Scikit-Optimize);</li> 
        <li> Apresentar uma tabela de comparação dos resultado da classificação considerando cada 1 dos otimizadores: acurácia de teste, kappa de teste e tempo médio do otimizador para fitar os dados. Dica: divisão treino é a sua escolha</li>    
</ol 

### Importando as bibliotecas

In [17]:
import pandas as pd
from scipy import stats

from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV

from sklearn.svm import SVC 


### Carregandos Dados

In [2]:
# Fonte:  https://archive.ics.uci.edu/ml/datasets/default+of+credit+card+clients
data = pd.read_csv('credit-card-default.csv')

## 1. Criando a função ``` feature_selector``` :

In [24]:
def feature_selector(lim,y_name):

    x = data.drop(y_name, axis=1)
    
    highly_corr_feat = data.columns[data.corr()[y_name].abs() > lim]
    highly_corr_feat = highly_corr_feat.drop(y_name)

    x_corr = x[highly_corr_feat]

    return x_corr

In [25]:
feature_selector(0.2,'default payment next month' )

,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5
0,2,2,-1,-1,-2
1,-1,2,0,0,0
2,0,0,0,0,0
3,0,0,0,0,0
4,-1,0,-1,0,0
...,...,...,...,...,...
29995,0,0,0,0,0
29996,-1,-1,-1,-1,0
29997,4,3,2,-1,0
29998,1,-1,0,0,0


## 2. Extrapolando a função ```get_best_model``` para abranger os classificadores svm e gxboost :

In [ ]:
def get_best_model_1(model_name, x, y):
    
    if model_name == 'd_tree':
        params = {'max_depth': [None, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
        model = DecisionTreeClassifier()
    elif model_name == 'r_forest':
        params = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 1, 3, 5, 7]}
        model = RandomForestClassifier()
    elif model_name == 'svm':
        params = {}
        model = SVC()
    elif model_name == 'gxboost':
        params = {}
        model = 
        
    else:
        print("Oops! That was no valid model. Try again...")
    
    
    grid = GridSearchCV(model,           
                        params,          
                        error_score=0.)  
    grid.fit(x, y) 
    
    print("Maior acurácia: {}".format(grid.best_score_))
    print("Melhores parâmetros: {}".format(grid.best_params_))
    print("Tempo médio para o otimizador fitar os dados (s): {}".format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    print("Tempo médio para o otimizador realizar a predição (s): {}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))

## 3. Extrpolando a função ```get_best_model``` para abranger a otimização Bayesiana (Scikit-Optimize):

In [35]:
# Bayes Search Scikit-learn
from skopt import BayesSearchCV

In [36]:
def get_best_model_2(model_name, opt_name, x, y):
    
    if model_name == 'd_tree':
        params = {'max_depth': [None, 1, 3, 5, 7, 9, 11, 13, 15, 17, 19, 21]}
        model = DecisionTreeClassifier()
    elif model_name == 'r_forest':
        params = {'n_estimators': [10, 50, 100, 200], 'max_depth': [None, 1, 3, 5, 7]}
        model = RandomForestClassifier()
    else:
        print("Oops! That was no valid model. Try again...")
    
    if opt_name == 'grid':
        grid = GridSearchCV(model,           
                            params,          
                            error_score=0.)  
        grid.fit(x, y) 
    elif opt_name == 'bayes':
        grid = BayesSearchCV(model, params)
        grid.fit(x,y)
    else:
        print("Oops! That was no valid optmization method. Try again...")
    
    print("Maior acurácia: {}".format(grid.best_score_))
    print("Melhores parâmetros: {}".format(grid.best_params_))
    print("Tempo médio para o otimizador fitar os dados (s): {}".format(round(grid.cv_results_['mean_fit_time'].mean(), 3)))
    print("Tempo médio para o otimizador realizar a predição (s): {}".format(round(grid.cv_results_['mean_score_time'].mean(), 3)))

In [38]:
y = data['default payment next month']
x_corr = feature_selector(0.2,'default payment next month' )
get_best_model_2('r_forest', "grid",x_corr, y)

Maior acurácia: 0.8211999999999999
Melhores parâmetros: {'max_depth': 3, 'n_estimators': 200}
Tempo médio para o otimizador fitar os dados (s): 0.969
Tempo médio para o otimizador realizar a predição (s): 0.093


## 4. Apresentando resultados: